The features we engineer were actually inspired from [this](https://www.kaggle.com/mhviraf/a-new-baseline-for-dsb-2019-catboost-model) kaggle kernel. The idea is to look at the various activities of a person before an assessment: such as the number and types of different clips they watched, the number and types of different games they played, the number and types of different assesments they took, their accuracy on these assessments and so on. 

Recall that specs.csv contains information that describes the various events in any game_session. We will not be using this to engineer our features. Thus, in order to save memory, we will not load specs.csv for the time being.

**List of features to create **


1) number of clips watched before attempting the assessment: Note that there are 20 different titles under the type 'clips'. We will create one entry for each such title. Additionally, one more entry for just the total number of clips watched (before the assessment) irrespective of their title. => 21 features.

2) number of activities done before the assessment: 8 different activities + 1 for all the activities = 9 features. 

3) number of games played before the assessment: 11 different games + 1 for all the games = 12 features

4) number of assessments done before the current assessment: 5 different assessments + 1 for all the assessments = 6 features

5) total (accumalated) amount of time spent watching different clips before the assessment: 20 different clips + 1 for all the clips = 21 features

6) total time spent on activities: 8 different activities + 1 for all the activities = 9 features

7) total time spent on games: 11 different games + 1 for all the games = 12

8) total time spent on previous assessments: 5 different assessments + 1 for all the assessments

9) average event_count in the clips watched (before the assessment): 20+1= 21 features

10) average event_count in activities (before the assessment): 8+1 = 9 features

11) average event_count in games (before the assessment): 11+1=12 features

12) average event_count in previous assessments: 5+1=6 features

13) average accuracy in previous assessments: 5+1 = 6 features


In total, we therefore have 48 x 3 + 6 = 150 features in the above list

Also note that we should only consider game_session which lasted for a non-zero time interval. Indeed there are game_session that were abandoned immediately after starting, for e.g. the session with id = '45bb1e1b6b50c07b'. These should not be included in the statistics being assembled above.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

In [2]:
print('The current working directory is: {}'.format(os.getcwd()))

The current working directory is: C:\Users\agarw\Dropbox\Kaggle data-science-bowl 2019\Prarit-data-science-bowl-2019


In [3]:
train=pd.read_csv('train.csv')

In [4]:
train.shape

(11341042, 11)

In [5]:
trainlbls=pd.read_csv('train_labels.csv')

In [6]:
trainlbls.shape

(17690, 7)

In [7]:
# merging the data in train and trainlbls according to game_session, installation_id and title
train=train.merge(trainlbls, how='left', on=['game_session', 'installation_id', 'title'])
train.shape

(11341042, 15)

In [8]:
train.keys()

Index(['event_id', 'game_session', 'timestamp', 'event_data',
       'installation_id', 'event_count', 'event_code', 'game_time', 'title',
       'type', 'world', 'num_correct', 'num_incorrect', 'accuracy',
       'accuracy_group'],
      dtype='object')

In [9]:
# for purposes of testing, we will first use only a small amount of data first. So we 
# choose to work with the data for only 10 installation ids. 

id_chosen=train.installation_id.unique()[0:10]

In [10]:
# the installation_ids chosen above
id_chosen

array(['0001e90f', '000447c4', '0006a69f', '0006c192', '0009a5a9',
       '0011edc8', '00129856', '0016b7cc', '00195df7', '001d0ed0'],
      dtype=object)

In [23]:
# reducing the train dataframe size by restricting to only 10 installation ids as chosen above
# use the pd.Series.isin() to choose rows based on a given list
# this was suggested in the following stackexchange post:
# https://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
train=train.loc[train.installation_id.isin(id_chosen)]

In [24]:
# shape of the reduced dataframe
train.shape

(11091, 15)

In [33]:
# converting the timestamp string to datetime format
# for documentation on pandas datetime functionality, see
# see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html 
train['datetime']=pd.to_datetime(train.timestamp)
train.drop(columns=['timestamp'], inplace=True)

In [53]:

train.head(10)

,event_id,game_session,event_data,installation_id,event_count,event_code,game_time,title,type,world,num_correct,num_incorrect,accuracy,accuracy_group,datetime
0,27253bdc,45bb1e1b6b50c07b,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,NaN,NaN,NaN,NaN,2019-09-06 17:53:46.937
1,27253bdc,17eeb7f223665f53,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:17.519
2,77261ab5,0848ef14a8dc6892,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:56.302
3,b2dba42b,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:56.387
4,1bb5fbdb,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:03.253
5,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:06.279
6,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:06.913
7,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":250,""y"":665,""stage_width"":...",0001e90f,6,4070,11255,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:07.546
8,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":279,""y"":629,""stage_width"":...",0001e90f,7,4070,11689,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:07.979
9,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":839,""y"":654,""stage_width"":...",0001e90f,8,4070,12272,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:08.566


In [46]:
min(train.datetime)

Timestamp('2019-08-06 04:57:18.904000')

In [45]:
train.datetime.min()

Timestamp('2019-08-06 04:57:18.904000')

In [40]:
max(train.datetime)

Timestamp('2019-10-11 15:03:37.812000')

In [44]:
pd.Timedelta(max(train.datetime)-min(train.datetime)).delta

5738778908000000

In [56]:
# function to compute the total time (in ms) spent in a game_session

def game_session_time(game_session=None, dataframe = train):
    if not game_session:
        print('no game_session was passed to the function')
        return 
        
    session_time= dataframe.loc[dataframe.game_session==game_session].datetime 
    start_time=session_time.min()
    end_time=session_time.max()
    time_interval=pd.Timedelta(end_time-start_time).delta/10**6
    
    return time_interval

In [48]:
game_session_time()

no game_session was passed to the function


In [57]:
game_session_time('0848ef14a8dc6892')

194307.0

In [54]:
train.loc[train.game_session=='0848ef14a8dc6892']

,event_id,game_session,event_data,installation_id,event_count,event_code,game_time,title,type,world,num_correct,num_incorrect,accuracy,accuracy_group,datetime
2,77261ab5,0848ef14a8dc6892,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:56.302
3,b2dba42b,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:56.387
4,1bb5fbdb,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:03.253
5,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:06.279
6,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:06.913
7,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":250,""y"":665,""stage_width"":...",0001e90f,6,4070,11255,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:07.546
8,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":279,""y"":629,""stage_width"":...",0001e90f,7,4070,11689,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:07.979
9,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":839,""y"":654,""stage_width"":...",0001e90f,8,4070,12272,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:08.566
10,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":807,""y"":619,""stage_width"":...",0001e90f,9,4070,12672,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:08.966
11,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":810,""y"":611,""stage_width"":...",0001e90f,10,4070,13389,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:09.673
